In [8]:
# 导入库
import numpy as np
import pandas as pd
from   datetime import datetime
from tqdm.notebook import tqdm
import os
import glob
import pysentiment2 as ps
import nltk
from   nltk.sentiment.vader import SentimentIntensityAnalyzer
from   IPython.core.display import HTML

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch

In [2]:
## 设置情感分析工具

# 下载Vader情感分析工具的词典
nltk.download('vader_lexicon')
# 初始化Loughran-McDonald情感分析模型
lm    = ps.LM()
# 初始化HIV-4情感分析模型
hiv4  = ps.HIV4()
# 初始化Vader情感分析模型
vader = SentimentIntensityAnalyzer()

# 如果有可用的第一块GPU则使用，否则使用CPU
device = torch.device("mps"if torch.backends.mps.is_available()else "cpu")

# 初始化FinBERT情感分析模型的tokenizer
finbert_tokenizer  = AutoTokenizer.from_pretrained("ProsusAI/finbert")
# 从FinBERT预训练模型加载模型
finbert_model      = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
# 配置FinBERT情感分析模型的运行环境（GPU or CPU）
finbert_classifier = pipeline("sentiment-analysis", model = finbert_model, tokenizer = finbert_tokenizer, device = device)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/wanghs/nltk_data...


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [16]:
def sentimentAnalysisLMD(text):
    tokens = lm.tokenize(text) #tokenize the text into relevant words in the LM dict
    score = lm.get_score(tokens) #score dict seperates into: NEG, POS, POLARITY, SUBJECTIVITY
    return score

def sentimentAnalysisHIV4(text):
    tokens = hiv4.tokenize(text)
    score = hiv4.get_score(tokens)
    return score

def sentimentAnalysisVader(df):
    df_vader = pd.DataFrame(df, columns=['Date', 'Text'])
    scores = [vader.polarity_scores(text) for text in tqdm(df_vader['Text'], desc="Processing with VADER")]
    scores_vader = pd.DataFrame(scores).add_prefix("vader_")
    df_vader = df_vader.join(scores_vader, rsuffix='_right')
    return df_vader['vader_compound']

def sentimentAnalysisFinBert(df):
    df_finbert = pd.DataFrame(df, columns=['Date', 'Text'])
    score = []
    for text in tqdm(df_finbert['Text'], desc="Processing with FinBERT"):
        classified = finbert_classifier(text, truncation=True)[0]
        if classified['label'] == "negative":
            score.append(classified['score']*(-1))
        elif classified['label'] == "positive":
            score.append(classified['score'])
        else:
            score.append(0)
    return score

In [ ]:
lst_files = []

Path = "../Data/Reuters_NER/*.csv"
count = 0

for fname in glob.glob(Path):
    lst_files.append(fname)

lst_files = sorted(lst_files)

for file in lst_files:
    file_name = os.path.basename(file)  # Get the file name from the full file path
    country_name = os.path.splitext(file_name)[0]  # Remove the file extension

    # 对单个国家的数据进行情感分析
    # if country_name == 'Finland':
    # Check if the file already exists
    if os.path.isfile('../Data/SentimentScores/{0}.csv'.format(country_name)):
        print("File already exists: {0}.csv".format(country_name))
    else:
        print(file)
        df = pd.read_csv(file)

        if df.shape[0] != 0:
            df = df.drop_duplicates(['Date','Headline'], keep='last')
            df = df.drop(['Unnamed: 0', 'Headline'], axis = 1)
            df['Text'] = df['Text'].astype(str)
            df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.date
            df.reset_index(drop=True, inplace=True)

            # LMD + HIV4
            df_LMD_HIV4 = pd.DataFrame(df, columns=['Date', 'Text'])
            # df_LMD_HIV4["scoreLMD"] = df_LMD_HIV4["Text"].apply(sentimentAnalysisLMD)
            # df_LMD_HIV4["scoreHIV4"] = df_LMD_HIV4["Text"].apply(sentimentAnalysisHIV4)
            # 使用tqdm在处理sentiment时显示进度条
            df_LMD_HIV4["scoreLMD"] = [sentimentAnalysisLMD(text) for text in tqdm(df_LMD_HIV4["Text"], desc="Processing with LMD")]
            df_LMD_HIV4["scoreHIV4"] = [sentimentAnalysisHIV4(text) for text in tqdm(df_LMD_HIV4["Text"], desc="Processing with HIV4")]
            df_LMD_HIV4 = pd.concat([df_LMD_HIV4.drop(["scoreLMD"], axis=1), df_LMD_HIV4["scoreLMD"].apply(pd.Series).add_prefix("LMD_")], axis=1)
            df_LMD_HIV4 = pd.concat([df_LMD_HIV4.drop(["scoreHIV4"], axis=1), df_LMD_HIV4["scoreHIV4"].apply(pd.Series).add_prefix("HIV4_")], axis=1)
            df[['LMD_Polarity', 'HIV4_Polarity']] = df_LMD_HIV4[['LMD_Polarity', 'HIV4_Polarity']]

            # VADER
            df['Vader_Polarity'] = sentimentAnalysisVader(df)

            # FINBERT
            df['FinBert_Polarity'] = sentimentAnalysisFinBert(df)

            # 储存一下df的原始数据
            df_original = df
            df['Country'] = country_name
            # Find the average Polarity for each Dictionary, per given date
            df = df.groupby('Date').mean(numeric_only=True).reset_index()
            df = df.sort_values(by = ['Date'], ascending = True)
            idx = pd.date_range('2012-01-01', '2022-12-31')
            df = df.set_index(['Date']).reindex(idx, fill_value=np.nan).rename_axis('Date').reset_index()

            df['Country'] = country_name
            df= df[['Date', 'Country', 'LMD_Polarity', 'HIV4_Polarity', 'Vader_Polarity', 'FinBert_Polarity']]

            count = count + 1
            print(count)
            df.to_csv('../Data/SentimentScores/{0}.csv'.format(country_name))
            # 保存df的原始数据
            df_original.to_csv('../Data/SentimentScores/{0}_original.csv'.format(country_name))


../Data/Reuters_NER/Australia.csv


Processing with LMD:   0%|          | 0/9234 [00:00<?, ?it/s]

Processing with HIV4:   0%|          | 0/9234 [00:00<?, ?it/s]

Processing with VADER:   0%|          | 0/9234 [00:00<?, ?it/s]

Processing with FinBERT:   0%|          | 0/9234 [00:00<?, ?it/s]

In [10]:
# 展示df的原始数据
df_original

,Date,Text,LMD_Polarity,HIV4_Polarity,Vader_Polarity,FinBert_Polarity
0,2021-06-01,Russia hold Poland to 1-1 draw in friendly. WR...,-0.333333,0.166667,0.7311,0.000000
1,2021-06-03,Bottas raised concerns over Mercedes' pit stop...,-0.888889,-0.085714,0.9036,0.000000
2,2021-06-08,Pfizer to test COVID-19 vaccine in larger grou...,-1.000000,0.400000,0.8402,0.000000
3,2021-06-08,U.S. to work with allies to secure electric ve...,-0.333333,0.411765,0.9865,0.000000
4,2021-06-09,Russia complains to UEFA over Ukraine kit with...,-0.555555,0.400000,0.9304,-0.820463
...,...,...,...,...,...,...
1324,2023-07-17,Nordic region's main bank Nordea's profit jump...,0.263158,0.581395,0.9853,0.896204
1325,2023-07-18,UN shipping agency's Council elects Panama's D...,-0.333333,0.361702,0.6698,0.000000
1326,2023-07-18,Finland's Gasum resumes LNG purchases from Rus...,-1.000000,0.285714,-0.1779,-0.821030
1327,2023-07-18,Singapore’s clean reputation gets a timely tes...,-0.837838,0.088608,-0.3628,0.000000


In [15]:
# 展示df最后5行的情感分析数据
df.tail()

,Date,Country,LMD_Polarity,HIV4_Polarity,Vader_Polarity,FinBert_Polarity
4013,2022-12-27,Finland,NaN,NaN,NaN,NaN
4014,2022-12-28,Finland,NaN,NaN,NaN,NaN
4015,2022-12-29,Finland,1.0,0.230769,0.7003,0.00000
4016,2022-12-30,Finland,-1.0,0.294118,-0.8360,-0.89182
4017,2022-12-31,Finland,NaN,NaN,NaN,NaN
